In [1]:
import time
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

file_path = '/Users/hejiaxuan/Desktop/American Express/TrainingData.csv'
full_df = pd.read_csv(file_path)

In [2]:
full_df .shape

(12229978, 71)

In [3]:
count_ind_recommended_1 = full_df['ind_recommended'].value_counts()[1]
count_ind_recommended_0 = full_df['ind_recommended'].value_counts()[0]
print("count_ind_recommended_1: ",count_ind_recommended_1 )
print("count_ind_recommended_0: ",count_ind_recommended_0 )


count_ind_recommended_1:  1547069
count_ind_recommended_0:  10682909


In [4]:
rec_1_percent = count_ind_recommended_1*1.0/(count_ind_recommended_1+count_ind_recommended_0)
print('rec_1_percent: ',rec_1_percent)   #推荐的比例

rec_0_percent = count_ind_recommended_0*1.0/(count_ind_recommended_1+count_ind_recommended_0)
print('rec_0_percent: ',rec_0_percent)   #没推荐的比例


rec_1_percent:  0.12649810163190808
rec_0_percent:  0.8735018983680919


In [5]:
p = count_ind_recommended_0/count_ind_recommended_1
p= round(p,3)
p

6.905

In [6]:
import numpy as np

# 原始列表
data = [1, -1, p, -p]

# 计算最大值和最小值
min_val = min(data)
max_val = max(data)

# 归一化
normalized_data = [(x - min_val) / (max_val - min_val) for x in data]

print(normalized_data)


[0.5724112961622013, 0.4275887038377987, 1.0, 0.0]


In [7]:
r1 = round(normalized_data[0],3)
r2 = round(normalized_data[1],3)
# r3 = 0
# r4 = 1

In [8]:
print(r1)
print(r2)

0.572
0.428


In [9]:
import pandas as pd


# 自定义函数，根据 activation 和 ind_recommended 列的取值计算 score
def calculate_score(row):
    if row['activation'] == 0 and row['ind_recommended'] == 0:
        return r1
    elif row['activation'] == 1 and row['ind_recommended'] == 0:
        return r2
    elif row['activation'] == 0 and row['ind_recommended'] == 1:
        return 0 
    elif row['activation'] == 1 and row['ind_recommended'] == 1:
        return 1
    else:
        return None

# 使用 apply() 方法将自定义函数应用到 DataFrame 中，并将结果存储到新的列 score 中
full_df['score'] = full_df.apply(calculate_score, axis=1)


In [10]:
full_df['score'].value_counts()

0.572    10622736
0.000     1537226
0.428       60173
1.000        9843
Name: score, dtype: int64

In [11]:
# mini_df = full_df.sample(frac=0.5)  # 50%的数据
mini_df = full_df 
del full_df


In [12]:
mini_df.shape

(6114989, 72)

In [13]:
mini_df_len = len(mini_df)

In [14]:
mini_df.head()

,ind_recommended,activation,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,...,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant,score
11236390,0,0,NaN,47.478571,49.475000,2.0,98.95,2.0,2.0,11.0,...,3140.0,0.916667,250.000000,40.904280,0.166667,230.0,3.916742,378293,383687,0.572
9008643,0,0,NaN,121.014192,33.262281,9.0,1895.95,57.0,47.0,37.0,...,301.0,0.450000,130.970000,99.266244,4.500000,103.0,1.772401,72242,466577,0.572
11588197,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16599.0,0.638889,123.000000,0.000000,0.000000,100.0,1.600396,236029,583290,0.572
8336124,0,0,NaN,47.562857,39.080784,8.0,1993.12,51.0,35.0,4.0,...,47637.0,1.000000,18.886818,99.865490,0.000000,64.0,5.290217,154451,214341,0.572
10838344,1,0,1.0,288.898017,25.705353,136.0,8020.07,312.0,215.0,12.0,...,27200.0,0.141026,38.457500,99.907814,37.833333,139.0,2.204629,404641,39304,0.000


In [15]:
file_path2 = '/Users/hejiaxuan/Desktop/American Express/EvaluationData.csv'
eval_df = pd.read_csv(file_path2)

In [16]:
# 添加缺失的列到 eval_df
eval_df['activation'] = 0  # 假设全部填充为 0
eval_df['ind_recommended'] = 0  # 假设全部填充为 0
eval_df['score'] = 0  # 假设全部填充为 0

In [17]:
eval_df.shape

(12604600, 72)

In [18]:
eval_df_len = len(eval_df)

In [19]:
combined_df = pd.concat([mini_df, eval_df], ignore_index=True)

In [20]:
del mini_df
del eval_df

In [21]:
combined_df.head()

,ind_recommended,activation,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,...,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant,score
0,0,0,NaN,47.478571,49.475000,2.0,98.95,2.0,2.0,11.0,...,3140.0,0.916667,250.000000,40.904280,0.166667,230.0,3.916742,378293,383687,0.572
1,0,0,NaN,121.014192,33.262281,9.0,1895.95,57.0,47.0,37.0,...,301.0,0.450000,130.970000,99.266244,4.500000,103.0,1.772401,72242,466577,0.572
2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16599.0,0.638889,123.000000,0.000000,0.000000,100.0,1.600396,236029,583290,0.572
3,0,0,NaN,47.562857,39.080784,8.0,1993.12,51.0,35.0,4.0,...,47637.0,1.000000,18.886818,99.865490,0.000000,64.0,5.290217,154451,214341,0.572
4,1,0,1.0,288.898017,25.705353,136.0,8020.07,312.0,215.0,12.0,...,27200.0,0.141026,38.457500,99.907814,37.833333,139.0,2.204629,404641,39304,0.000


In [22]:
combined_df.shape

(18719589, 72)

## Data Prepocessing

In [23]:
combined_df.columns

Index(['ind_recommended', 'activation', 'customer_digital_activity_04',
       'customer_spend_01', 'customer_industry_spend_01',
       'customer_industry_spend_02', 'customer_industry_spend_03',
       'customer_industry_spend_04', 'customer_industry_spend_05',
       'customer_spend_02', 'customer_spend_03', 'customer_merchant_02',
       'customer_merchant_01', 'customer_spend_04', 'customer_spend_05',
       'customer_spend_06', 'customer_spend_07', 'merchant_spend_01',
       'merchant_spend_02', 'merchant_spend_03', 'merchant_spend_04',
       'merchant_spend_05', 'merchant_spend_06', 'merchant_spend_07',
       'merchant_spend_08', 'merchant_profile_01', 'customer_merchant_03',
       'customer_profile_01', 'customer_profile_02',
       'customer_digital_activity_05', 'customer_spend_13',
       'customer_digital_activity_06', 'customer_spend_14',
       'customer_digital_activity_07', 'customer_digital_activity_08',
       'customer_digital_activity_09', 'customer_digital_acti

处理空值

归一化

把分类变量编码

### 1.处理空值

In [24]:

# 设置显示选项，使其显示所有行和列
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 计算每列的空值数量
null_counts = combined_df.isnull().sum()

# 计算每列的空值比例
null_percentages = (null_counts / len(combined_df)) * 100


# 创建一个新的数据框来存储结果
statistic_df = pd.DataFrame({'Null Counts': null_counts, 'Null Percentage': null_percentages})

# 计算每列的最大值
max_values = combined_df.max()

# 计算每列的最小值
min_values = combined_df.min()

# 统计每列负数的个数
negative_counts = (combined_df < 0).sum()


statistic_df['Max Value'] = max_values
statistic_df['Min Value'] = min_values
statistic_df['Negative Counts'] = negative_counts

In [25]:
statistic_df

,Null Counts,Null Percentage,Max Value,Min Value,Negative Counts
ind_recommended,0,0.000000,1.000000e+00,0.000000,0
activation,0,0.000000,1.000000e+00,0.000000,0
customer_digital_activity_04,16114847,86.085474,8.560000e+02,1.000000,0
customer_spend_01,2935985,15.684025,2.489723e+05,1.000000,0
customer_industry_spend_01,8961951,47.874721,2.222939e+05,0.010000,0
customer_industry_spend_02,8961951,47.874721,1.521000e+03,1.000000,0
customer_industry_spend_03,8961951,47.874721,1.889311e+07,0.010000,0
customer_industry_spend_04,8961951,47.874721,4.494000e+04,1.000000,0
customer_industry_spend_05,8961951,47.874721,3.600000e+02,1.000000,0
customer_spend_02,2935985,15.684025,1.520000e+02,0.000000,0


In [26]:
statistic_df.shape

(72, 5)

In [27]:
# 找到空值比例大于70%的列名
columns_to_drop = statistic_df[statistic_df['Null Percentage'] > 70].index

In [28]:
columns_to_drop 

Index(['customer_digital_activity_04', 'customer_merchant_02',
       'customer_merchant_01', 'customer_spend_14',
       'customer_digital_activity_07', 'customer_digital_activity_08',
       'customer_digital_activity_09', 'customer_spend_15',
       'customer_spend_17', 'merchant_spend_11',
       'customer_digital_activity_18', 'customer_digital_activity_19'],
      dtype='object')

In [29]:
'merchant_profile_01' in columns_to_drop   #分类变量列

False

In [30]:
# 'ind_recommended' in columns_to_drop   

In [31]:
# 'activation' in columns_to_drop   

In [32]:
# 从df中删除这些空值比例大于70%的列
combined_df.drop(columns=columns_to_drop, inplace=True)


In [33]:
combined_df.shape

(18719589, 60)

In [34]:
combined_df.columns

Index(['ind_recommended', 'activation', 'customer_spend_01',
       'customer_industry_spend_01', 'customer_industry_spend_02',
       'customer_industry_spend_03', 'customer_industry_spend_04',
       'customer_industry_spend_05', 'customer_spend_02', 'customer_spend_03',
       'customer_spend_04', 'customer_spend_05', 'customer_spend_06',
       'customer_spend_07', 'merchant_spend_01', 'merchant_spend_02',
       'merchant_spend_03', 'merchant_spend_04', 'merchant_spend_05',
       'merchant_spend_06', 'merchant_spend_07', 'merchant_spend_08',
       'merchant_profile_01', 'customer_merchant_03', 'customer_profile_01',
       'customer_profile_02', 'customer_digital_activity_05',
       'customer_spend_13', 'customer_digital_activity_06',
       'customer_digital_activity_10', 'customer_digital_activity_11',
       'customer_digital_activity_12', 'customer_digital_activity_13',
       'customer_digital_activity_14', 'customer_digital_activity_15',
       'customer_digital_activity_

In [35]:
combined_df_columns_1 = combined_df.columns

剩下的之中，除了'ind_recommended', 'activation','merchant_profile_01' 列

In [36]:
# # 提取除了 'ind_recommended', 'activation', 'merchant_profile_01' 列之外的所有列
# columns_to_fill = combined_df.columns.difference(['ind_recommended', 'activation', 'merchant_profile_01'])

# # 填充空值为 0
# combined_df[columns_to_fill] = combined_df[columns_to_fill].fillna(0)


In [37]:
# 检查 `Min Value` 为 0 或 1 的列的缺失值并填充，但不处理 'ind_recommended', 'activation', 'merchant_profile_01' 这三列
for column in statistic_df.index:
    if column not in columns_to_drop and column in combined_df.columns and column not in columns_to_keep:
        if statistic_df.loc[column, 'Min Value'] == 0:
            combined_df[column].fillna(0, inplace=True)
        elif statistic_df.loc[column, 'Min Value'] == 1:
            combined_df[column].fillna(1, inplace=True)
        else:
            # 使用众数填充
            mode_value = combined_df[column].mode()[0]
            combined_df[column].fillna(mode_value, inplace=True)

### 2. 标准化（除了'ind_recommended', 'activation','merchant_profile_01'之外）

In [38]:
from sklearn.preprocessing import StandardScaler

# 创建 StandardScaler 对象
scaler = StandardScaler()

# 定义需要标准化的列
columns_to_standardize = combined_df.columns.difference(['score','ind_recommended', 'activation', 'merchant_profile_01','customer', 'merchant'])

# 对除了指定列之外的所有列进行标准化
combined_df[columns_to_standardize] = scaler.fit_transform(combined_df[columns_to_standardize])


### 3. 编码 分类变量（对'merchant_profile_01'）

In [39]:
combined_df.shape

(18719589, 60)

In [40]:
combined_df['merchant_profile_01'].unique()

array([202., 419., 101., 415.,  nan, 403., 103., 102., 420., 324., 326.,
       322., 417., 416., 314., 319., 406., 339., 307., 413., 333., 412.,
       405., 320., 503., 408., 317., 418., 501., 402., 201., 414., 411.,
       303., 410., 306., 401., 311., 407., 330., 337., 315., 507., 335.,
       308., 331., 313., 329., 502., 328., 309., 409., 504., 312., 327.,
       332., 338., 301., 321., 340., 325., 342., 316., 304., 203., 506.,
       302., 305., 336., 323., 334., 404., 318., 310., 341., 505.])

In [41]:
combined_df['merchant_profile_01'].isna().sum()

1507284

In [42]:
combined_df['merchant_profile_01'].describe()

count    1.721230e+07
mean     2.601828e+02
std      1.384917e+02
min      1.010000e+02
25%      1.010000e+02
50%      3.190000e+02
75%      4.060000e+02
max      5.070000e+02
Name: merchant_profile_01, dtype: float64

In [43]:
# 对分类变量进行独热编码   75列?
encoded_df = pd.get_dummies(combined_df['merchant_profile_01'], prefix='merchant_profile_01')

In [44]:
encoded_df.shape

(18719589, 75)

In [45]:
# 删除原始的 merchant_profile_01 列
combined_df.drop('merchant_profile_01', axis=1, inplace=True)

# 将编码后的数据框与原始数据框合并
combined_df = pd.concat([combined_df, encoded_df], axis=1)

In [46]:
combined_df.head(5)

,ind_recommended,activation,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,customer_spend_03,customer_spend_04,customer_spend_05,customer_spend_06,customer_spend_07,merchant_spend_01,merchant_spend_02,merchant_spend_03,merchant_spend_04,merchant_spend_05,merchant_spend_06,merchant_spend_07,merchant_spend_08,customer_merchant_03,customer_profile_01,customer_profile_02,customer_digital_activity_05,customer_spend_13,customer_digital_activity_06,customer_digital_activity_10,customer_digital_activity_11,customer_digital_activity_12,customer_digital_activity_13,customer_digital_activity_14,customer_digital_activity_15,customer_digital_activity_16,customer_spend_16,customer_digital_activity_17,customer_digital_activity_03,distance_01,customer_digital_activity_20,distance_02,distance_03,customer_spend_18,customer_spend_19,customer_digital_activity_21,customer_digital_activity_22,distance_04,merchant_profile_02,merchant_spend_09,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant,score,merchant_profile_01_101.0,merchant_profile_01_102.0,merchant_profile_01_103.0,merchant_profile_01_201.0,merchant_profile_01_202.0,merchant_profile_01_203.0,merchant_profile_01_301.0,merchant_profile_01_302.0,merchant_profile_01_303.0,merchant_profile_01_304.0,merchant_profile_01_305.0,merchant_profile_01_306.0,merchant_profile_01_307.0,merchant_profile_01_308.0,merchant_profile_01_309.0,merchant_profile_01_310.0,merchant_profile_01_311.0,merchant_profile_01_312.0,merchant_profile_01_313.0,merchant_profile_01_314.0,merchant_profile_01_315.0,merchant_profile_01_316.0,merchant_profile_01_317.0,merchant_profile_01_318.0,merchant_profile_01_319.0,merchant_profile_01_320.0,merchant_profile_01_321.0,merchant_profile_01_322.0,merchant_profile_01_323.0,merchant_profile_01_324.0,merchant_profile_01_325.0,merchant_profile_01_326.0,merchant_profile_01_327.0,merchant_profile_01_328.0,merchant_profile_01_329.0,merchant_profile_01_330.0,merchant_profile_01_331.0,merchant_profile_01_332.0,merchant_profile_01_333.0,merchant_profile_01_334.0,merchant_profile_01_335.0,merchant_profile_01_336.0,merchant_profile_01_337.0,merchant_profile_01_338.0,merchant_profile_01_339.0,merchant_profile_01_340.0,merchant_profile_01_341.0,merchant_profile_01_342.0,merchant_profile_01_401.0,merchant_profile_01_402.0,merchant_profile_01_403.0,merchant_profile_01_404.0,merchant_profile_01_405.0,merchant_profile_01_406.0,merchant_profile_01_407.0,merchant_profile_01_408.0,merchant_profile_01_409.0,merchant_profile_01_410.0,merchant_profile_01_411.0,merchant_profile_01_412.0,merchant_profile_01_413.0,merchant_profile_01_414.0,merchant_profile_01_415.0,merchant_profile_01_416.0,merchant_profile_01_417.0,merchant_profile_01_418.0,merchant_profile_01_419.0,merchant_profile_01_420.0,merchant_profile_01_501.0,merchant_profile_01_502.0,merchant_profile_01_503.0,merchant_profile_01_504.0,merchant_profile_01_505.0,merchant_profile_01_506.0,merchant_profile_01_507.0
0,0,0,-0.117899,-0.099540,-0.319524,-0.084688,-0.261870,-0.360016,0.567009,0.140496,0.083015,-0.147222,-0.007911,0.359270,0.100843,-0.193707,0.202014,-0.005280,-0.001795,0.434740,-0.091655,-0.132985,0.186893,-0.119161,-0.123103,-0.188309,0.608191,-0.338119,-0.360227,-0.454133,-0.443711,-0.597520,-0.513693,-0.630211,-0.769041,2.177169,-0.355668,-0.256093,-0.105738,2.074642,0.064144,-0.137042,-0.069855,-0.062876,-0.342038,-0.055369,-0.134449,0.948589,-0.723595,-0.691167,1.196321,0.026093,-0.201633,-0.411781,0.347349,-0.140050,378293,383687,0.572,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,-0.017090,-0.120953,0.095828,-0.018399,0.674138,1.075429,4.158282,4.001965,-0.016900,0.789384,2.597817,2.248231,0.

In [47]:
mini_df = combined_df.iloc[:mini_df_len]
eval_df = combined_df.iloc[mini_df_len:]

In [48]:
# 删除 'activation' 和 'ind_recommended','score' 列
eval_df.drop(['activation', 'ind_recommended','score'], axis=1, inplace=True)

/var/folders/rl/m5k47t5116g5vwgk1kj6j98h0000gn/T/ipykernel_15672/1109138367.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_df.drop(['activation', 'ind_recommended','score'], axis=1, inplace=True)


In [49]:
# 打印 mini_df 和 eval_df 的形状
print("mini_df shape:", mini_df.shape)
print("eval_df shape:", eval_df.shape)

mini_df shape: (6114989, 134)
eval_df shape: (12604600, 131)


In [50]:
mini_df['customer'].head()

0    378293
1     72242
2    236029
3    154451
4    404641
Name: customer, dtype: int64

In [51]:
eval_df['customer'].head()

6114989    467915
6114990    467915
6114991    467915
6114992    467915
6114993    467915
Name: customer, dtype: int64

In [52]:
del combined_df

## Model

In [53]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [54]:
# 分离特征和标签
X = mini_df.drop(['activation', 'ind_recommended','score'], axis=1)
y = mini_df['score']

# # 对目标变量 y 进行归一化
# scaler = MinMaxScaler(feature_range=(0, 1))  # 设置归一化的范围为 [0, 1]
# y_normalized = scaler.fit_transform(y.values.reshape(-1, 1))  

# 分割数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=66)


In [55]:
del mini_df,X,y

In [56]:
# 创建XGBoost兼容的数据结构
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)


In [57]:
# 设置XGBoost参数
param = {
    'max_depth': 8,  # 决策树的最大深度
    'eta': 0.5,      # 学习率
    'objective': 'binary:logistic',  # 二元分类的逻辑回归问题
    'nthread': 4     # CPU线程数
}



In [61]:
# bst = xgb.train(param, dtrain, num_round)

In [58]:
num_round = 100  # 训练轮数
watchlist = [(dtest, 'eval'), (dtrain, 'train')]
# 训练模型
# bst = xgb.train(param, dtrain, num_round)
bst = xgb.train(param, dtrain, num_round, watchlist, early_stopping_rounds=50)


/Users/hejiaxuan/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:617: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[0]	eval-logloss:0.68866	train-logloss:0.68860
[1]	eval-logloss:0.68730	train-logloss:0.68717
[2]	eval-logloss:0.68680	train-logloss:0.68662
[3]	eval-logloss:0.68658	train-logloss:0.68635
[4]	eval-logloss:0.68644	train-logloss:0.68617
[5]	eval-logloss:0.68636	train-logloss:0.68604
[6]	eval-logloss:0.68633	train-logloss:0.68598
[7]	eval-logloss:0.68629	train-logloss:0.68591
[8]	eval-logloss:0.68626	train-logloss:0.68584
[9]	eval-logloss:0.68622	train-logloss:0.68577
[10]	eval-logloss:0.68621	train-logloss:0.68574
[11]	eval-logloss:0.68619	train-logloss:0.68569
[12]	eval-logloss:0.68616	train-logloss:0.68562
[13]	eval-logloss:0.68615	train-logloss:0.68557
[14]	eval-logloss:0.68615	train-logloss:0.68553
[15]	eval-logloss:0.68613	train-logloss:0.68547
[16]	eval-logloss:0.68612	train-logloss:0.68545
[17]	eval-logloss:0.68611	train-logloss:0.68540
[18]	eval-logloss:0.68611	train-logloss:0.68539
[19]	eval-logloss:0.68610	train-logloss:0.68534
[20]	eval-logloss:0.68609	train-logloss:0.68528
[2

In [64]:
# # 设置XGBoost参数
# param2 = {
#     'max_depth': 8,  # 决策树的最大深度
#     'eta': 0.5,      # 学习率
#     'objective': 'binary:logistic',  # 二元分类的逻辑回归问题
#     'nthread': 4     # CPU线程数
# }
# num_round = 100  # 训练轮数，这里设定一个较大的值
# watchlist = [(dtest, 'eval'), (dtrain, 'train')]
# # 训练模型，并设置早停法
# bst2 = xgb.train(param2, dtrain, num_round, watchlist, early_stopping_rounds=50)


[0]	eval-logloss:0.32167	train-logloss:0.32149
[1]	eval-logloss:0.18280	train-logloss:0.18249
[2]	eval-logloss:0.11321	train-logloss:0.11276
[3]	eval-logloss:0.07536	train-logloss:0.07477
[4]	eval-logloss:0.05405	train-logloss:0.05333
[5]	eval-logloss:0.04184	train-logloss:0.04097
[6]	eval-logloss:0.03483	train-logloss:0.03381
[7]	eval-logloss:0.03081	train-logloss:0.02963
[8]	eval-logloss:0.02852	train-logloss:0.02718
[9]	eval-logloss:0.02720	train-logloss:0.02571
[10]	eval-logloss:0.02646	train-logloss:0.02482
[11]	eval-logloss:0.02604	train-logloss:0.02427
[12]	eval-logloss:0.02581	train-logloss:0.02389
[13]	eval-logloss:0.02566	train-logloss:0.02358
[14]	eval-logloss:0.02556	train-logloss:0.02333
[15]	eval-logloss:0.02551	train-logloss:0.02315
[16]	eval-logloss:0.02550	train-logloss:0.02301
[17]	eval-logloss:0.02547	train-logloss:0.02289
[18]	eval-logloss:0.02546	train-logloss:0.02276
[19]	eval-logloss:0.02544	train-logloss:0.02266
[20]	eval-logloss:0.02543	train-logloss:0.02248
[2

In [62]:
# 保存模型
bst.save_model('xgb_score1.txt')

In [53]:
# # 预测测试集
# preds = bst.predict(dtest)

In [61]:
# from sklearn.metrics import roc_auc_score
# preds = bst.predict(dtest)
# auc = roc_auc_score(y_test, preds)
# print(f'AUC: {auc}')


In [54]:
# # 设置一个阈值，通常是0.5
# threshold = 0.5
# # 将概率转换为类别标签
# pred_labels = (preds >= threshold).astype('int')

# # 评估模型性能
# from sklearn.metrics import accuracy_score, roc_auc_score

# accuracy = accuracy_score(y_test, pred_labels)
# auc = roc_auc_score(y_test, preds)

# print(f'Accuracy: {accuracy}')
# print(f'AUC: {auc}')

Accuracy: 0.9941733347070069
AUC: 0.9327851830517798


## prediction

In [70]:
# eval_df.drop(['score'], axis=1, inplace=True)


In [63]:
eval_df.shape

(12604600, 131)

In [64]:
eval_df.head()

,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,customer_spend_03,customer_spend_04,customer_spend_05,customer_spend_06,customer_spend_07,merchant_spend_01,merchant_spend_02,merchant_spend_03,merchant_spend_04,merchant_spend_05,merchant_spend_06,merchant_spend_07,merchant_spend_08,customer_merchant_03,customer_profile_01,customer_profile_02,customer_digital_activity_05,customer_spend_13,customer_digital_activity_06,customer_digital_activity_10,customer_digital_activity_11,customer_digital_activity_12,customer_digital_activity_13,customer_digital_activity_14,customer_digital_activity_15,customer_digital_activity_16,customer_spend_16,customer_digital_activity_17,customer_digital_activity_03,distance_01,customer_digital_activity_20,distance_02,distance_03,customer_spend_18,customer_spend_19,customer_digital_activity_21,customer_digital_activity_22,distance_04,merchant_profile_02,merchant_spend_09,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant,merchant_profile_01_101.0,merchant_profile_01_102.0,merchant_profile_01_103.0,merchant_profile_01_201.0,merchant_profile_01_202.0,merchant_profile_01_203.0,merchant_profile_01_301.0,merchant_profile_01_302.0,merchant_profile_01_303.0,merchant_profile_01_304.0,merchant_profile_01_305.0,merchant_profile_01_306.0,merchant_profile_01_307.0,merchant_profile_01_308.0,merchant_profile_01_309.0,merchant_profile_01_310.0,merchant_profile_01_311.0,merchant_profile_01_312.0,merchant_profile_01_313.0,merchant_profile_01_314.0,merchant_profile_01_315.0,merchant_profile_01_316.0,merchant_profile_01_317.0,merchant_profile_01_318.0,merchant_profile_01_319.0,merchant_profile_01_320.0,merchant_profile_01_321.0,merchant_profile_01_322.0,merchant_profile_01_323.0,merchant_profile_01_324.0,merchant_profile_01_325.0,merchant_profile_01_326.0,merchant_profile_01_327.0,merchant_profile_01_328.0,merchant_profile_01_329.0,merchant_profile_01_330.0,merchant_profile_01_331.0,merchant_profile_01_332.0,merchant_profile_01_333.0,merchant_profile_01_334.0,merchant_profile_01_335.0,merchant_profile_01_336.0,merchant_profile_01_337.0,merchant_profile_01_338.0,merchant_profile_01_339.0,merchant_profile_01_340.0,merchant_profile_01_341.0,merchant_profile_01_342.0,merchant_profile_01_401.0,merchant_profile_01_402.0,merchant_profile_01_403.0,merchant_profile_01_404.0,merchant_profile_01_405.0,merchant_profile_01_406.0,merchant_profile_01_407.0,merchant_profile_01_408.0,merchant_profile_01_409.0,merchant_profile_01_410.0,merchant_profile_01_411.0,merchant_profile_01_412.0,merchant_profile_01_413.0,merchant_profile_01_414.0,merchant_profile_01_415.0,merchant_profile_01_416.0,merchant_profile_01_417.0,merchant_profile_01_418.0,merchant_profile_01_419.0,merchant_profile_01_420.0,merchant_profile_01_501.0,merchant_profile_01_502.0,merchant_profile_01_503.0,merchant_profile_01_504.0,merchant_profile_01_505.0,merchant_profile_01_506.0,merchant_profile_01_507.0
6114989,-0.028990,-0.058494,0.095828,-0.052681,-0.091687,-0.104826,-0.399872,-0.041459,0.451119,-0.166577,-0.305525,-0.428639,-0.147388,-0.146825,-0.074315,0.069883,0.029744,0.488909,-0.141973,-0.111847,-0.970168,-0.127432,-0.115955,-0.188309,-0.509316,0.086682,0.087491,0.225312,0.212826,0.452418,0.363363,0.492691,1.091045,-0.412459,-0.355668,-0.256093,NaN,-0.609930,-0.270763,-0.148526,-0.006737,0.043153,-0.342038,-0.126786,-0.148790,1.076479,-0.081862,-0.644245,0.877451,-0.168604,0.668984,0.055181,1.679689,-0.502430,467915,599167,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6114990,-0.028990,NaN,-0.378860,NaN,-0.278888,-0.391915,-0.399872,-0.041459,-0.234414,-0.166577,-0.305525,-0.428639,1.220454,-0.205427,0.362375,-0.033466,-0.024322,0.

In [65]:
# 使用模型对新的测试集进行预测
dtest2 = xgb.DMatrix(eval_df)

# 使用加载的模型进行预测
preds = bst.predict(dtest2)

In [66]:
# 预测结果添加到 DataFrame 中
eval_df['predicted_score'] = preds 
# 只保留特定列
eval_result = eval_df[['customer', 'merchant', 'predicted_score']]


In [68]:
eval_result['predicted_score'].describe()

count    1.260460e+07
mean     4.670925e-01
std      9.993121e-02
min      9.302403e-04
25%      4.495697e-01
50%      5.052580e-01
75%      5.290480e-01
max      9.739905e-01
Name: predicted_score, dtype: float64

In [59]:
# eval_result['predicted_score'].value_counts()

In [67]:
# 将 DataFrame 保存为 CSV 文件，不保存索引列
eval_result.to_csv('eval_predictions_xgboost_0318_score.csv', index=False)

In [69]:
eval_result.shape

(12604600, 3)

In [70]:
eval_result.head()

,customer,merchant,predicted_score
6114989,467915,599167,0.493097
6114990,467915,686617,0.433463
6114991,467915,829193,0.427028
6114992,467915,1077034,0.392878
6114993,467915,876647,0.406561
